In [ ]:
import DeepFMKit.core as dfm
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

def generate_and_verify_noise(
    asd_at_1hz=1000.0,
    num_points=200000,
    fs=200000.0
):
    """
    Generates and verifies a stream of laser frequency noise.

    This script uses the DeepFitFramework to generate a time series of
    laser frequency noise with a 1/f characteristic (random walk). It then
    calculates the Amplitude Spectral Density (ASD) of the generated noise
    using Welch's method and plots it on a log-log scale to verify that
    it follows the expected 1/f slope.

    Parameters
    ----------
    asd_at_1hz : float, optional
        The target Amplitude Spectral Density of the frequency noise at 1 Hz,
        in units of Hz/sqrt(Hz). This defines the magnitude of the noise.
    num_points : int, optional
        The total number of samples to generate in the time series.
    fs : float, optional
        The sampling frequency in Hz.
    """
    print("=" * 60)
    print("Generating and Verifying Laser Frequency Noise")
    print(f"Target ASD @ 1 Hz: {asd_at_1hz:.1f} Hz/sqrt(Hz)")
    print(f"Sampling Frequency: {fs / 1e3:.1f} kHz")
    print(f"Number of Points: {num_points}")
    print("=" * 60)

    # --- 1. Setup Simulation using the Framework ---
    # Instantiate the main framework object
    dff = dfm.DeepFitFramework()

    # Create a new simulation configuration
    dff.new_sim("noise_test")
    sim_config = dff.sims["noise_test"]

    # Configure the simulation for noise generation
    sim_config.f_samp = fs
    # Set the desired laser frequency noise ASD.
    # All other noise sources default to 0.
    sim_config.f_n = asd_at_1hz

    # --- 2. Generate the Noise Time Series ---
    # The time_axis argument is only used to determine the number of samples.
    time_axis_placeholder = np.arange(num_points)
    print("Generating noise time series...")
    noise_dict = dff._generate_noise_arrays(sim_config, time_axis_placeholder)
    frequency_noise_t = noise_dict['laser_frequency']
    print("Noise generation complete.")

    # --- 3. Calculate the Amplitude Spectral Density (ASD) ---
    print("Calculating ASD using Welch's method...")
    # Use Welch's method for a robust PSD estimate.
    # nperseg defines the length of each segment, which determines frequency resolution.
    # Using fs as the segment length gives a frequency resolution of 1 Hz.
    freqs, psd = signal.welch(
        frequency_noise_t,
        fs=fs,
        window='hann',
        nperseg=int(fs),
        scaling='density'
    )
    # ASD is the square root of the PSD
    asd = np.sqrt(psd)
    print("ASD calculation complete.")

    # --- 4. Plot Verification ---
    # Create the theoretical 1/f reference line
    # The line must pass through (1 Hz, asd_at_1hz)
    theoretical_asd = asd_at_1hz / freqs

    fig, ax = plt.subplots(figsize=(10, 7))

    ax.loglog(freqs, asd, label='Generated Noise ASD')
    ax.loglog(
        freqs,
        theoretical_asd,
        'r--',
        linewidth=2,
        label=r'Theoretical $1/f$ Slope'
    )

    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel(r'Frequency Noise ASD (Hz / $\sqrt{\rm Hz}$)')
    ax.set_title('Verification of Generated Laser Frequency Noise')
    ax.grid(True, which='both', linestyle=':')
    ax.legend()
    ax.set_ylim(bottom=np.min(asd[asd > 0])/2) # Ensure y-axis starts reasonably

    plt.tight_layout()
    plt.show()

# --- Run the script with example parameters ---
if __name__ == "__main__":
    generate_and_verify_noise(
        asd_at_1hz=1000.0,
        num_points=1e6, # Use more points for better low-frequency stats
        fs=200e3
    )